In [ ]:
!pip install -q unsloth==2025.4.7 datasets==3.5.1

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
from unsloth import FastLanguageModel


MAX_SEQ_LENGTH = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    dtype=torch.bfloat16,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "up_proj",
        "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 42,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.673 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.4.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [3]:
model.print_trainable_parameters()

trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039


In [4]:
from datasets import load_dataset

dataset_name = "5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca"
SYS_INSTRUCT = "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."
raw_dataset = load_dataset(dataset_name, split="train")

def convert_to_chat_format(conversations):
    messages = [{"role": "system", "content": SYS_INSTRUCT}]
    for msg in conversations:
        role = "user" if msg["from"] == "human" else "assistant"
        messages.append({"role": role, "content": msg["value"]})
    return messages

def format_prompt(example):
    messages = convert_to_chat_format(example["conversations"])
    return {
        "text": tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
    }

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding="max_length",
    )

dataset = raw_dataset.map(format_prompt, remove_columns=raw_dataset.column_names)
dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    save_total_limit=4,
    logging_steps=20,
    output_dir="./checkpoint/llama3-1b-multi-conversation",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    save_strategy="steps",
    save_steps=50,
    report_to="none",
    remove_unused_columns=True,
    max_steps=400,
    bf16=True,
)

trainer=SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,697 | Num Epochs = 5 | Total steps = 400
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
100,1.527100
200,1.349700
300,1.299000
400,1.277400


TrainOutput(global_step=400, training_loss=1.3632887840270995, metrics={'train_runtime': 12874.6447, 'train_samples_per_second': 3.977, 'train_steps_per_second': 0.031, 'total_flos': 6.14358673287807e+17, 'train_loss': 1.3632887840270995})

In [ ]:
# load model from ckpt dir then save to huggingface hub
from unsloth import FastLanguageModel
import torch


model, tokenizer = FastLanguageModel.from_pretrained(
    "./checkpoint/llama3-1b-multi-conversation/checkpoint-400",
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=torch.bfloat16,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 3. Max memory: 23.673 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.4.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [2]:
type(model)

peft.peft_model.PeftModelForCausalLM

In [3]:
model.push_to_hub_merged(
    "thuanan/Llama-3.2-1B-Instruct-Chat-sft",
    commit_message="Merge weights to push to hub",
)

Unsloth: You're not saving a tokenizer as well?
You can do it separately via `tokenizer.push_to_hub(...)`
Unsloth: You are pushing to hub, but you passed your HF username = thuanan.
We shall truncate thuanan/Llama-3.2-1B-Instruct-Chat-sft to Llama-3.2-1B-Instruct-Chat-sft


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 83.71 out of 125.4 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 60.50it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/thuanan/Llama-3.2-1B-Instruct-Chat-sft


In [4]:
type(tokenizer)

transformers.tokenization_utils_fast.PreTrainedTokenizerFast

In [5]:
tokenizer.push_to_hub(
    "thuanan/Llama-3.2-1B-Instruct-Chat-sft",
    commit_message="Push tokenizer to hub",
)

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [9]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_new_tokens=128,
    temperature=1.0,
    do_sample=False,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.3
)


prompt = [
    {
        "role": "system",
        "content": "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."
    },
    {
        "role": "user",
        "content": """Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."""
    },
    {
        "role": "assistant",
        "content": """Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề."""
    },
    {
        "role": "user",
        "content": """Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?"""
    }
]

chat_text = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    tokenize=False
)

inputs = tokenizer(
    chat_text,
    return_tensors="pt"
).to("cuda:0")

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        generation_config=generation_config,
    )
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

if "assistant\n" in output_text:
    answer = output_text.split("assistant\n")[-1].strip()
else:
    answer = output_text.strip()

print("Assistant reply:", answer)

Assistant reply: Chắc chắn! Dưới đây là một vài mẹo về cách các nhà tổ chức và cầu thủ team có thể áp dụng cho trò chơi sắp tới: - Đảm bảo rằng tất cả những người tham gia đều được đào tạo đầy đủ trước khi bắt đầu cuộc đua hoặc giải trí mới nhất. Điều quan trọng cần lưu tâm là việc chuẩn bị kỹ lưỡng sẽ giúp giảm thiểu rủi ro xảy ra sự cố trên sân khấu cũng như đảm bảo mọi thứ diễn ra suôn sẻ với tư duy sáng suốt tốt. - Tìm kiếm phản hồi từ đồng nghiệp và giám sát từng bước hành động của bạn nhằm xác định
